In [1]:
from keras.datasets import mnist
from utils.DOSELM import DOSELM
from utils.DRRN import DRRN
import numpy as np
import torch
import time as t
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

In [2]:
def evaluation_summary(description, predictions, true_labels):
    print("Evaluation for: " + description)
    precision = precision_score(predictions, true_labels, average="macro")
    recall = recall_score(predictions, true_labels, average="macro")
    accuracy = accuracy_score(predictions, true_labels)
    f1 = fbeta_score(predictions, true_labels, beta=1, average="macro")
    print("Classifier '%s' has Acc=%0.3f P=%0.3f R=%0.3f F1=%0.3f" % (description,accuracy,precision,recall,f1))
    print(classification_report(predictions, true_labels, digits=3))
    print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [3]:
def scale(X):
    return (X - X.mean())/(X.std()*10)

In [4]:
def splitdata(x, y, splits=2, first=-1):
    length = x.shape[0]

    out = []
    if first == -1:
        splitlen = length // splits
        for i in range(splits):
            out.append((x[splitlen*i:splitlen*(i+1)], y[splitlen*i:splitlen*(i+1)]))
    else:
        splits-= 1
        split1 = int(length*first)
        splitn = (length-split1)//splits
        out.append((x[0:split1], y[0:split1]))
        for i in range(splits):
            a = split1+splitn*i
            b = split1+splitn*(i+1)
            z = (x[a:b], y[a:b])
            out.append(z)
    return out

In [5]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = scale(np.reshape(train_X.ravel(), (-1,784)))
test_X = scale(np.reshape(test_X.ravel(), (-1,784)))

y = np.zeros([60000,10])

for i in range(len(y)):
    for j in range(len(y[i])):
        if j == train_y[i]:
            y[i,j] = 1
        else:
            y[i,j] = 0
            
train_y=y

In [6]:
# torch.manual_seed(6519)  # this seed scores really badly!
# torch.manual_seed(666)

device = torch.device("cpu")
drrn = DRRN(784, 10, 100, subnets=1, device=device)

X = torch.from_numpy(train_X).float()
y = torch.from_numpy(train_y).float()

result = drrn.train(X, y)
tX = torch.from_numpy(test_X).float()
pred = drrn.predict(tX)
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

evaluation_summary('ELM', pred_arg, test_y)

Evaluation for: ELM
Classifier 'ELM' has Acc=0.801 P=0.798 R=0.805 F1=0.797
              precision    recall  f1-score   support

         0.0      0.900     0.871     0.885      1013
         1.0      0.927     0.872     0.899      1206
         2.0      0.792     0.788     0.790      1037
         3.0      0.851     0.690     0.762      1246
         4.0      0.814     0.829     0.821       964
         5.0      0.713     0.711     0.712       895
         6.0      0.816     0.880     0.847       889
         7.0      0.876     0.748     0.807      1205
         8.0      0.597     0.844     0.699       688
         9.0      0.693     0.816     0.749       857

    accuracy                          0.801     10000
   macro avg      0.798     0.805     0.797     10000
weighted avg      0.812     0.801     0.803     10000


Confusion matrix:
 [[ 882    2   11   12    5   23   17    5   21    2]
 [   0 1052   13   37    2    9    3   12    7    0]
 [  18   35  817   58    8   15   21   

In [7]:
torch.manual_seed(0)

device = torch.device("cpu")
doselm = DOSELM(784, 10, [250], device=device)
split_data = splitdata(train_X, train_y, 2)

train_start = t.time()
j = 0

for i in split_data:
    X = torch.from_numpy(i[0]).float()
    y = torch.from_numpy(i[1]).float()
    result = doselm.train(X,y)
    j+=1
    if j==0:
        break
train_end = t.time()

tX = torch.from_numpy(test_X).float()

pred_start = t.time()
pred = doselm.predict(tX)
pred_end = t.time()
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

evaluation_summary('ELM', pred_arg, test_y)

torch.Size([784, 250])


RuntimeError: svd_cpu: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated singular values (error code: 14).

In [ ]:
torch.manual_seed(0)

device = torch.device("cpu")
doselm = DOSELM(784, 10, [1000], device=device)

X = torch.from_numpy(train_X).float()[:60000]
y = torch.from_numpy(train_y).float()[:60000]

result = doselm.train(X, y)
tX = torch.from_numpy(test_X).float()
pred = doselm.predict(tX)
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

evaluation_summary('ELM', pred_arg, test_y)